# Regression model on handaxe dataset

This example is adapted from Lewis (2017) Chapter 6

In this example, we build a linear regression model to predict the maximum length of a Lower Paleolithic hand axe.

The Handaxes data-frame contains 600 measurements on Lower Paleolithic hand-axes from Furze Platt, Berkshire, England.

## Libraries and dataset

In [ ]:
library(data.table) # to handle the data in a more convenient manner
library(tidyverse) # for a better work flow and more tools to wrangle and visualize the data
library(plotly) # for interactive visualizations
library(psych) # for visualizing relationship among pairs of variables
library(GGally) # for better visualizing relationship among pairs of variables
library(corrplot) # for correlation plots
library(listviewer) # for visualizing nested data structures
library(caret) # for detecting highly correlated variables
library(broom) # for getting a glance of model fit
library(TSrepr) # for evaluating predictive power
library(archdata) # for handaxe data
library(lindia) # for regression model diagnostics
library(lmtest) # for variance, auto-correlation, and coefficient significance tests
library(car) # for variance and multi-collinearity tests
library(mctest) # for multi-collinearity tests
library(ppcor) # for partial correlations
library(fit.models) # for comparing models
library(MASS) # for robust regression
library(forecast) # for auto-correlation plots
options(warn = -1) # for suppressing messages

In [ ]:
data("Handaxes",package="archdata")

In [ ]:
?Handaxes

```
Format
A data frame with 600 observations on the following 8 variables.

Catalog
Specimen catalog number

L
Maximum Length

L1
Distance from the butt to the location of the maximum breadth measured along the length dimension

B
Maximum breadth

B1
Breadth measured at 1/5 of the length from the tip. Measured perpendicular to the length

B2
Breadth measured at 1/5 of the length from the butt. Measured perpendicular to the length

T
Maximum thickness, not necessarily measured at the maximum breadth

T1
Thickness measured at B1
```

L variable will be our dependent variable

In [ ]:
handaxes_dt <- as.data.table(Handaxes)

## Explore data

In [ ]:
str(handaxes_dt)

Summarize numeric variables:

In [ ]:
handaxes_dt %>% purrr::keep(is.numeric) %>% sapply(quantile) %>% t()

In [ ]:
handaxes_dt %>% purrr::keep(is.numeric) %>% # select columns
    tidyr::gather() %>% # reshape into long format in columns "key" and "value"
    ggplot(aes(value)) + # plot value
        facet_wrap(~ key, scale = "free" ) + # divide into separate plots by key
        geom_density(fill = "green")  # get density plots

In [ ]:
handaxes_dt %>% purrr::keep(is.numeric) %>% psych::pairs.panels()

In [ ]:
handaxes_dt %>% purrr::keep(is.numeric) %>% GGally::ggpairs()

In [ ]:
handaxes_dt %>% purrr::keep(is.numeric) %>% cor() %>%

corrplot::corrplot.mixed(upper = "ellipse",
                         lower = "number",
                         tl.pos = "lt",
                         number.cex = .5,
                         lower.col = "black",
                         tl.cex = 0.7)

All variables are normally distributed. Some of the dependent variables are highly correlated as B and B2, L1 and B1 pairs

## Data wrangling

### Log transformation

In [ ]:
y <- handaxes_dt[,log(L)]

In [ ]:
data_sample <- handaxes_dt[,log(.SD), .SDcols = L1:T1]

### Addressing multi-collinearity

We can address multi-collinearity by removing highly correlated variables

In [ ]:
cor_matrix <- data_sample %>% cor()

In [ ]:
rid_col <- caret::findCorrelation(cor_matrix, cutoff = 0.6, exact = F)
rid_col

In [ ]:
remove_vars <- colnames(data_sample)[rid_col]
remove_vars

In [ ]:
data_sample2 <- data_sample[,-c(remove_vars), with = F]
data_sample2

## Partition data

In [ ]:
set.seed(2016)
train_ind <- data_sample2[,sample(.I, 550)]

In [ ]:
y_train <- y[train_ind]
y_test <- y[-train_ind]

In [ ]:
data_train <- data_sample2[train_ind]
data_test <- data_sample2[-train_ind]

In [ ]:
data_train[,.N]
data_test[,.N]

## Train model

**EXERCISE 1:**

Create a model called "fit1" using all the variables in train data

**SOLUTION 1:**

In [ ]:
fit1 <- lm( y_train ~., data = data_train )

## Evaluate model

### Summary of model

In [ ]:
fit1_sum <- summary(fit1)

In [ ]:
listviewer::jsonedit(fit1, mode = "form")

In [ ]:
listviewer::jsonedit(fit1_sum, mode = "form")

### Inspect coefficients

In [ ]:
fit1_sum$coefficients

The minus sign on T1's coefficient is weird since the correlation between T1 and L1 is positive!

We drop it from the model

In [ ]:
fit2 <- lm(y_train ~ L1 + B2 + T, data = data_train)

In [ ]:
fit2_sum <- summary(fit2)

In [ ]:
fit2_sum$coefficients

All variables are significant(ly different than 0) as per their p values

### Glance of model fit

In [ ]:
broom::glance(fit2)

F statistic is highly significant. However r squared value could be higher

### Multi-collinearity

Variance Inflation Factor (VIF) is a measure for detecting multi-collinearity

> The general rule of thumb is that VIFs exceeding 4 warrant further investigation, while VIFs exceeding 10 are signs of serious multicollinearity requiring correction

(https://onlinecourses.science.psu.edu/stat501/node/347/)

In [ ]:
car::vif(fit2)

No VIF value in the model is sufficiently high for us to be alert for multicollinearity 

An alternative method to check for multicollinearity is Overall Multicollinearity Diagnostics from mctest package:

In [ ]:
mctest::omcdiag(data_train[,.(L1, B2, T)], y_train)

And for further inspection we will use All Individual Multicollinearity Diagnostics Result:

In [ ]:
mctest::imcdiag(data_train[,.(L1, B2, T)], y_train)

According to the output, we should not care for multi-collinearity in any of the individual variables

- Finally, for examining the pattern of multicollinearity, it is required to conduct t-test for correlation coefficient.

- We’ll use the ‘ppcor’ package to compute the partial correlation coefficients along with the t-statistic and corresponding p-values.

In [ ]:
ppcor::pcor(data_train[,.(L1, B2, T)], method = "pearson")

There is a significant partial correlation between L1 and T variables. However since VIF and other multi-collinearity diagnostics did not alert us for a problematic case, we do not exclude any of these variables.

Partial correlations will be used as a confirmatory test in case significant multi-collinearity is detected in previous tests 

### Distribution of residuals

We may assess the normality of residuals

In [ ]:
fit2$residuals %>% as.data.table()

In [ ]:
fit2$residuals %>% as.data.table() %>%
ggplot(aes(.)) + # plot value
    geom_density(fill = "green")  # get density plots

And a qq plot will confirm the normality:

In [ ]:
lindia::gg_qqplot(fit2, scale.factor = 1)

In a Normal Q-Q plot the ordered residuals (y-axis) are plotted against the expected quantiles from a standard normal distribution function (x-axis).

The plotted points should lie on an upward sloping (45 degree) straight line.
qqplot shows the residual points fall approximately along a straight upward sloping line

And last we can conduct a Shapiro-Wilk normality test on residuals:

In [ ]:
shapiro.test(fit2$residuals)

The null hypothesis is that the residuals are from a normal distribution.

The p-value of the test statistic, at 0.81, is rather large (maximum value =1), and we cannot reject the null hy-
pothesis.

### Heteroskedasticity: Is variance constant?

Heteroskedasticity (non-constant variance) causes the estimated standard error of the estimates to be wrong. This means
the confidence intervals and hypotheses tests may not be reliable. The scale-location plot is useful for assessing the constantvariance assumption.

You can view the plot by setting which=3 in the plot function or gg_scalelocation function from the lindia package.

In [ ]:
lindia::gg_scalelocation(fit2)

There should be no discernible pattern to the plot with the points spread evenly around a horizontal line.

Looking closely at the figure, the points seem to form a homogeneous cloud, equally spread on both sides of the line. This is good.

However, there is a slight upward slope to the line, this is a little worrying.

We will conduct two tests for non-constant variance:

In [ ]:
lmtest::bptest(fit2)

In [ ]:
car::ncvTest(fit2)

For both statistical tests the p-value is less than 0.01, and we reject the null hypothesis of constant variance

Heteroskedasticity impacts the standard errors, not the estimated values.

Our real concern is to ensure the coefficients in the model are statistically significant (they probably are given their tiny original p-values, but we should double check).

A popular solution for heteroscedasticity, involves adjusting the standard errors to better refl
ect the actual underlying probabilities for hypothesis testing.

The car package has the function hccm that does the job. It calculates the heteroscedasticity-corrected covariance matrix from which the standard errors are derived.

We combine it with coeftest function from the lmtest package to recalculate the significance of coefficients:

In [ ]:
lmtest::coeftest(fit2, vcov = car::hccm(fit2))

So, given the level of heteroskedasticity, the coefficients are still significant

### Influential observations

We want to check for influential observations. These are observations that greatly impact the slope of the regression line. 

A good way to identify problematic data points is via leverage.

The leverage of an observation measures how far away it is from the other observations. It takes values between 0 and 1.

A point with zero leverage has no effect on the regression model.

In [ ]:
lev <- fit2 %>% model.matrix() %>% hat()
summary(lev)

In this case, the largest value is relatively small, at 0.04. To identify this point:

In [ ]:
which.max(lev)
data_train[which.max(lev)]

The influencePlot from car package also shows influential observations:

In [ ]:
car::influencePlot(fit2)

You may have noticed that the influencePlot function also reports Cook’s distance, this metric identifies points which have
more influence than other points. Influence is the amount that a data point is affecting the regression line, measured by how
much the regression line would change if the point were excluded from the regression model.

Points with a large Cook’s distance have a high influence, and might require further investigation.

A visual representation of Cook’s distance is obtained via:

In [ ]:
lindia::gg_cooksd(fit2)

The points identified in Figure 6.7, or Figure 6.8 are not necessarily definitive.

The key is whether excluding these points will have a significant impact on the regression coefficients.

In our case, we have identified a handful of potential observations, none of which appear particularly extreme.

### Robust regression

One way to assess whether our intuition is correct, is to reestimate the model using robust regression, and then check to
see whether the coefficient estimates are similar.

If they are, then we can leave keep the identified examples in our analysis.

If not, you would re-run your regression model, dropping each potential influential observation one at a time, and then assess the impact on the coefficients.

Observations that significantly impact the regression coefficients can be excluded from further analysis.

```
Robust regression is an alternative to least squares regression when data are contaminated with outliers or influential observations, and it can also be used for the purpose of detecting influential observations.
```
(https://stats.idre.ucla.edu/r/dae/robust-regression/)

The rlm function in the MASS package estimates robust regression.

We will also use the fit.models function to display both regular and robust regression results:

In [ ]:
fit.models::fmclass.add.class("lmfm", "rlm")

fm1 <- fit.models::fit.models(c("rlm", "lm"),
                              y_train ~ L1 + B2 + T,
                              data = data_train)

In [ ]:
fm1

A quick visual inspection of the estimated coefficient does not real any significant differences between regular regression (lm) and robust regression (rlm).

We could test for a difference statistically if we saw a large difference, but won’t bother in this case.

It appears the identified “outlying”observations are not sufficiently different from the underlying sample for us to remove
them from our analysis.

### Autocorrelation

In [ ]:
forecast::ggAcf(fit2$residuals)

We see, there are spikes.

We can check autocorrelation with Durbin-Watson test

In [ ]:
lmtest::dwtest(y_train ~ L1 + B2 + T, data = data_train)

The p-value, at 0.63, indicates there is no evidence of correlatedresiduals.

It confirms our earlier observation.

## Predictive power

**EXERCISE 2:**

Assess the predictive power of the model on the test set. You may use rmse and mae measures using TSrepr package, or scatterplots or any other method you would like from the first example